In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
!pip install -U ultralytics

In [7]:
from ultralytics import YOLO
# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

In [8]:
# Use the model
#model.train(data="coco128.yaml", epochs=3)  # train the model
#metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
print(results)

100%|██████████| 476k/476k [00:00<00:00, 20.8MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 244.9ms
Speed: 25.0ms preprocess, 244.9ms inference, 3527.4ms postprocess per image at shape (1, 3, 640, 480)


In [9]:
!pip install onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.1 MB/s eta 0:00:00


In [10]:
import onnx
import onnxruntime
import torch.onnx

In [17]:
# Конвертация модели в формат ONNX
#torch.onnx.export(model, "bus.jpg", "yolov8.onnx")
success = model.export(format='onnx')

Ultralytics YOLOv8.2.23 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 0.8s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (2.5s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


In [18]:
!ls /content

bus.jpg  datasets  runs  sample_data  yolov8n.onnx  yolov8n.pt


In [22]:
onnx_model_path='/content/yolov8n.onnx'
onnx_model=onnx.load(onnx_model_path)

In [23]:
# Создание исполнителя ONNX-модели
ort_session = onnxruntime.InferenceSession(onnx_model_path)

In [31]:
#https://dev.to/andreygermanov/how-to-create-yolov8-based-object-detection-web-service-using-python-julia-nodejs-javascript-go-and-rust-4o8e - делал по этому гайду
from PIL import Image
img = Image.open("bus.jpg")
img_width, img_height = img.size;
img = img.resize((640,640))
img = img.convert("RGB");

In [33]:
import numpy as np
input = np.array(img)
input.shape
input = input.transpose(2,0,1)
input.shape
input = input.reshape(1,3,640,640)
input = input.astype(np.float32)

In [34]:
# Запуск модели из файла ONNX
ort_inputs = {ort_session.get_inputs()[0].name: input}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели Yolo8 в ONNX:", ort_outs)

Результат запуска модели Yolo8 в ONNX: [array([[[     6.6988,      10.805,      25.342, ...,      535.39,      548.06,      561.05],
        [     3.3887,      2.9973,      3.4545, ...,      566.58,      576.25,      605.07],
        [     12.244,      17.849,      21.825, ...,      210.03,      183.66,      158.33],
        ...,
        [          0,           0,           0, ...,  2.9802e-08,  2.9802e-07,  1.2517e-06],
        [          0,           0,           0, ...,  9.8348e-07,  1.9372e-06,  3.2485e-06],
        [          0,           0,           0, ...,  4.7684e-07,   1.967e-06,  4.5002e-06]]], dtype=float32)]


In [35]:
!pip install onnx2torch
from onnx2torch.converter import convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.4 MB/s eta 0:00:00


In [36]:
torch_model = convert(onnx_model_path)
input = torch.from_numpy(input)
output = torch_model(input)
print(output)

tensor([[[6.6989e+00, 1.0805e+01, 2.5342e+01,  ..., 5.3539e+02, 5.4806e+02, 5.6105e+02],
         [3.3887e+00, 2.9973e+00, 3.4545e+00,  ..., 5.6658e+02, 5.7625e+02, 6.0507e+02],
         [1.2244e+01, 1.7849e+01, 2.1825e+01,  ..., 2.1003e+02, 1.8366e+02, 1.5833e+02],
         ...,
         [3.2687e-15, 2.7802e-16, 1.2164e-14,  ..., 4.3387e-08, 3.0332e-07, 1.2041e-06],
         [1.9300e-09, 9.4084e-11, 4.5921e-09,  ..., 9.7548e-07, 1.9234e-06, 3.2217e-06],
         [1.3999e-11, 1.8535e-12, 3.3871e-11,  ..., 5.0513e-07, 2.0161e-06, 4.4900e-06]]], grad_fn=<CatBackward0>)
